In [1]:
import numpy as np
import pandas as pd
import os
import EOS_extension_fixPlin
import TOVsolver
import matplotlib.pyplot as plt

EOS_og = pd.read_table('data/EOSCEFTVE1.dat', header=None).to_numpy()

keep = EOS_og[:,0] < 0.1601

EOS_start = EOS_og[keep,:]

In [2]:
def progress_statement(i, total):
    if int(i*100/total) == 25 and int((i-1)*100/total) != 25:
        print("The run is 25% complete")
        done_25=True
    elif int(i*100/total) == 50 and int((i-1)*100/total) != 50:
        print("The run is 50% complete")
        done_50=True
    elif int(i*100/total) == 75 and int((i-1)*100/total) != 75:
        print("The run is 75% complete")
        done_75=True
    
def filter_MRL(MRL_table, m_ub=5, r_lb=7):
    raw_mass = MRL_table[:,0]
    raw_radius = MRL_table[:,1]
    raw_Lambda = MRL_table[:,2]
    # create boolean arrays to test if points are good
    m2big = raw_mass < m_ub
    r2small = raw_radius > r_lb
    # the bool array of points we will keep
    keep = np.logical_and(m2big, r2small)
    # define new arrays we will keep, this filters out erronious points in array
    radius = raw_radius[keep]
    mass = raw_mass[keep]
    Lambda = raw_Lambda[keep]
    
    leng = len(radius) # get number of physical points
    MRL = np.zeros((leng, 3)) # initialize MRL table
    MRL[:,0], MRL[:,1], MRL[:,2] = mass, radius, Lambda # put into table
    
    return MRL

def exclude_by_mass(MRL, maxm_lb=1.8, maxm_ub=4.1):
    maxm = np.max(MRL[:,0])
    if maxm < maxm_ub and maxm > maxm_lb:
        MRL_passes = True
    else:
        MRL_passes = False
    return MRL_passes

def mkdirs(nsamp, ext_type, newsavename, datapath):
    if newsavename==None:
        EOSdir_name = ext_type+str(nsamp)+'EOS'
        MRLdir_name = ext_type+str(nsamp)+'MRL'
    else:
        EOSdir_name = ext_type+str(nsamp)+'EOS'+newsavename
        MRLdir_name = ext_type+str(nsamp)+'MRL'+newsavename
    
    if not(EOSdir_name in os.listdir(datapath)) and not(MRLdir_name in os.listdir(datapath)):
        os.makedirs(datapath+EOSdir_name)
        os.makedirs(datapath+MRLdir_name)
        
    return EOSdir_name, MRLdir_name

def get_num_eos(dirpath):
    numlist = []
    for file in os.listdir(dirpath):
        if '.dat' in file:
            numlist.append(int(file[:-4]))
    
    if len(numlist) == 0:
        num_eos = 0
    else:
        num_eos = max(numlist)
        
    return num_eos
    

def generate_tables(n_tables, EOS_start, nsamp_EOS, ext_type, MRL_size=100, maxm_thresh=1.8, newsavename=None):
    datapath = os.getcwd() + "/data/" #where the data is stored
    # make new directories with proper names
    EOSdir_name, MRLdir_name = mkdirs(nsamp, ext_type, newsavename, datapath)
    # returns number where
    filenumstart = get_num_eos(datapath+EOSdir_name) + 1
    i = 0
    num_failed = 0
    
    # extend EOS and store parameters
    while i < n_tables:
        
        if ext_type == 'poly':
            EOS, ns, gammas, Ks = EOS_extension_fixPlin.extend(EOS_start, nsamp=nsamp_EOS,
                                                            ext_type=ext_type, max_gamma=9)
            param_string = "ns =" + str(ns) + ' gammas =' + str(gammas) + ' Ks =' + str(Ks)
        else:
            EOS, ns, cs = EOS_extension_fixPlin.extend(EOS_start, nsamp=nsamp_EOS, ext_type=ext_type)
            param_string = "ns =" + str(ns) + ' cs =' + str(cs)
        
        MRL_table = TOVsolver.solve(EOS, MRL_size) #solve tov
        # filter out erronious points
        MRL = filter_MRL(MRL_table)
        #check if maximum mass is realistic
        MRL_inrange = exclude_by_mass(MRL, maxm_lb=maxm_thresh)
        
        if MRL_inrange == False:
            num_failed += 1
            i -= 1
        else:
            EOSname = datapath + EOSdir_name + '/' + str(filenumstart + i) + '.dat' # make names for file
            MRLname = datapath + MRLdir_name + '/' + str(filenumstart + i) + '.dat'
            np.savetxt(EOSname, EOS, header=param_string) # save files
            np.savetxt(MRLname, MRL)
        
        progress_statement(i, total=n_tables)
        i +=1
        
    print(str(num_failed)+' iterations failed for '+str(n_tables)+' successful tables.')

In [20]:
ntables = 500
ext_type = "lin"
nsamp = 15
newsavename = None

generate_tables(ntables, EOS_og, nsamp, ext_type, newsavename=newsavename)

The run is 25% complete
The run is 50% complete
The run is 75% complete
58 iterations failed for 500 successful tables.


In [21]:
ntables = 500
ext_type = "lin"
nsamp = 15
newsavename = None

generate_tables(ntables, EOS_og, nsamp, ext_type, newsavename=newsavename)

The run is 25% complete
The run is 50% complete
The run is 75% complete
58 iterations failed for 500 successful tables.


In [24]:
ext_type = "lin"
nsamp = 15
newsavename = None
# to run single instances and specify name
saveindex = 723

datapath = os.getcwd() + "/data/" #where the data is stored
# make new directories with proper names
EOSdir_name, MRLdir_name = mkdirs(nsamp, ext_type, newsavename, datapath)

EOS, ns, cs = EOS_extension_fixPlin.extend(EOS_og, nsamp=8, ext_type='lin')
param_string = "ns =" + str(ns) + ' cs =' + str(cs)
MRL_table = TOVsolver.solve(EOS, size=100) #solve tov
# filter out erronious points
MRL = filter_MRL(MRL_table)
#check if maximum mass is realistic
MRL_inrange = exclude_by_mass(MRL, maxm_lb=1.8)

if MRL_inrange == True:
    EOSname = datapath + EOSdir_name + '/' + str(saveindex) + '.dat' # make names for file
    MRLname = datapath + MRLdir_name + '/' + str(saveindex) + '.dat'
    np.savetxt(EOSname, EOS, header=param_string) # save files
    np.savetxt(MRLname, MRL)
    print("saving file...")

saving file...
